In [ ]:
from keycloak import KeycloakAdmin, KeycloakOpenIDConnection

In [ ]:
# INIT INSTANCE KeycloakOpenIDConnection

conn = KeycloakOpenIDConnection(
    server_url="http://0.0.0.0:8180",
    username="admin",
    password="admin",
    realm_name="master",
    client_id="account",
    client_secret_key="CAG72F9OGK5HwDo57uU0SshGDm4iGf0b",
)

In [ ]:
# INIT INSTANCE KeycloakAdmin

kc_admin = KeycloakAdmin(connection=conn)

In [ ]:
import string
import random
import json


def generate_id_number():
    allowed_chars = string.ascii_uppercase + string.digits
    unique_chars = random.sample(
        population=[char for char in allowed_chars if char not in "O0LI"], k=8
    )
    random.shuffle(unique_chars)
    matricule = "-".join(
        [
            "".join(unique_chars[:2]),
            "".join(unique_chars[2:5]),
            "".join(unique_chars[5:]),
        ]
    )
    return matricule


payload_user = {
    "username": "maxence.doe@pm.me",
    "email": "maxence.doe@pm.me",
    "firstName": "John",
    "lastName": "Doe",
    "enabled": True,
    "emailVerified": True,
    "credentials": [{"type": "password", "value": "p@55word"}],
    "attributes": {
        "id_number": generate_id_number(),
        "phone_number": "+2250151571396",
        "primary": True,
        "app_users": True,
    },
}

new_user = await kc_admin.a_create_user(payload=payload_user)
new_user

In [75]:
# GET ALL USERS

users = await kc_admin.a_get_users({"first": 1, "max": 10})
users

[{'id': '25a8654f-b42b-424e-90a3-3c8fb18d4484',
  'username': 'dimitri.doe@pm.me',
  'firstName': 'John',
  'lastName': 'Doe',
  'email': 'dimitridoe@pm.me',
  'emailVerified': True,
  'attributes': {'id_number': ['Q6-UVN-HD8'],
   'phone_number': ['+2250151571396'],
   'app_users': ['true'],
   'primary': ['true']},
  'createdTimestamp': 1722953556562,
  'enabled': True,
  'totp': False,
  'disableableCredentialTypes': [],
  'requiredActions': [],
  'notBefore': 0,
  'access': {'manageGroupMembership': True,
   'view': True,
   'mapRoles': True,
   'impersonate': True,
   'manage': True}},
 {'id': 'c0b71a82-3815-4231-bdc7-376fcf0c5d72',
  'username': 'john.doe@pm.me',
  'firstName': 'John',
  'lastName': 'Doe',
  'email': 'johndoe@pm.me',
  'emailVerified': False,
  'attributes': {'id_number': ['7R-VCU-Y4E'],
   'phone_number': ['+2250151571396'],
   'app_users': ['true'],
   'primary': ['true']},
  'createdTimestamp': 1722952981905,
  'enabled': True,
  'totp': False,
  'disableableC

In [ ]:
# GET COUNT USERS

users_count = await kc_admin.a_users_count({})
users_count

In [ ]:
# GET USER

user = await kc_admin.a_get_user(user_id=new_user)
user

In [ ]:
# CREATE GROUP

import string
import random
from slugify import slugify


for _ in range(1, 5):
    c = random.choice(string.ascii_uppercase)
    group_name = f"agent rencenseur groupe {c}".upper()
    payload = {"name": group_name, "attributes": {"slug": [slugify(group_name)]}}

    new_group = await kc_admin.a_create_group(payload=payload, skip_exists=True)
    new_group

In [147]:
# GET ALL GROUPS

groups = await kc_admin.a_get_groups({})
groups

[{'id': 'a7c0d334-0d5b-4263-985a-91e08d4125a1',
  'name': 'AGENT RENCENSEUR GROUPE A',
  'path': '/AGENT RENCENSEUR GROUPE A',
  'subGroupCount': 0,
  'subGroups': [],
  'access': {'view': True,
   'viewMembers': True,
   'manageMembers': True,
   'manage': True,
   'manageMembership': True}},
 {'id': 'c4eb6924-5f92-4c84-b388-66636c069aae',
  'name': 'AGENT RENCENSEUR GROUPE C',
  'path': '/AGENT RENCENSEUR GROUPE C',
  'subGroupCount': 0,
  'subGroups': [],
  'access': {'view': True,
   'viewMembers': True,
   'manageMembers': True,
   'manage': True,
   'manageMembership': True}},
 {'id': '963defeb-0d34-4979-ada1-b2d1667819af',
  'name': 'AGENT RENCENSEUR GROUPE E',
  'path': '/AGENT RENCENSEUR GROUPE E',
  'subGroupCount': 0,
  'subGroups': [],
  'access': {'view': True,
   'viewMembers': True,
   'manageMembers': True,
   'manage': True,
   'manageMembership': True}},
 {'id': '4cbefd17-52de-48c6-8996-7b4d34761b6d',
  'name': 'AGENT RENCENSEUR GROUPE G',
  'path': '/AGENT RENCENSEUR

In [148]:
# GET GROUP

sliced_groups = groups[3:]
group_id = sliced_groups[0].get("id")
group = await kc_admin.a_get_group(group_id=group_id)
group

{'id': '4cbefd17-52de-48c6-8996-7b4d34761b6d',
 'name': 'AGENT RENCENSEUR GROUPE G',
 'path': '/AGENT RENCENSEUR GROUPE G',
 'subGroupCount': 0,
 'subGroups': [],
 'attributes': {'slug': ['agent-rencenseur-groupe-g']},
 'realmRoles': [],
 'clientRoles': {},
 'access': {'view': True,
  'viewMembers': True,
  'manageMembers': True,
  'manage': True,
  'manageMembership': True}}

In [149]:
# ASSIGN GROUP TO USER

user_id = users[0].get("id")
user_groups = await kc_admin.a_get_user_groups(user_id=user_id)

if user_groups:
    await kc_admin.a_group_user_remove(
        user_id=user_id, group_id=user_groups[0].get("id")
    )

result = await kc_admin.a_group_user_add(user_id=user_id, group_id=group["id"])
result

{}

In [150]:
# GET USER GROUPS

groups = await kc_admin.a_get_user_groups(user_id=user_id)
groups

[{'id': '4cbefd17-52de-48c6-8996-7b4d34761b6d',
  'name': 'AGENT RENCENSEUR GROUPE G',
  'path': '/AGENT RENCENSEUR GROUPE G',
  'subGroups': []}]